# CNNによるチェックマーク認識

In [63]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import cv2

In [64]:
import json
import torch
from torchvision import transforms
from PIL import Image
from torch.utils.data import Dataset, DataLoader
import requests

In [65]:
# 画像をグレースケールでロード
image = cv2.imread('../sheet1.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

## 方針
ニューラルネットモデルを利用した、チェックボックスの検出とチェックマークの有無を判定する。
が、方針として２通りがあると思われる。
* ボックス検知を行うモデル、検出したボックスのチェックマークを判定するモデルの2つを作成する方法
* ボックス検知とチェックマーク判定を同時に行う方法
    * チェックマークを学習させ、画像中のチェックマークを検知させる方法

## 画像中のチェックボックスを検出するモデルの作成


In [66]:
#チェックボックスの検出モデルを作成
# CNNアーキテクチャを定義
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # 入力画像チャネル3、出力チャネル6、カーネルサイズ5
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 2)  # チェックマークと非チェックマークの2クラスを想定

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [67]:
class CustomDataSet(Dataset):
    def __init__(self, json_file, transform=None):
        self.data = []
        with open(json_file, 'r') as f:
            for line in f:
                self.data.append(json.loads(line))
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        image = Image.open(requests.get(item['data_row']['row_data'], stream=True).raw)
        objects = item['projects'][list(item['projects'].keys())[0]]['labels'][0]['annotations']['objects']
        boxes = []
        labels = []
        for obj in objects:
            bbox = obj['bounding_box']
            boxes.append([bbox['top'], bbox['left'], bbox['top'] + bbox['height'], bbox['left'] + bbox['width']])
            labels.append(1 if obj['name'] == 'checked-box' else 0)  # Assuming 'checked-box' is label 1 and 'nonchecked-box' is label 0
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        image_id = torch.tensor([idx])
        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["image_id"] = image_id
        if self.transform is not None:
            image, target = self.transform(image, target)
        return image, target



In [68]:
with open('../export-result.ndjson', 'r') as f:
    for i, line in enumerate(f):
        print(json.loads(line))
        if i >= 2:  # Print the first 3 lines
            break

{'data_row': {'id': 'clkc190n608wj077n2xeq09sq', 'external_id': 'sheet1.jpg', 'row_data': 'https://storage.labelbox.com/clkc0y32f0hhh071s37v9c8xv%2F53b20677-2bb2-5323-dbc5-532ef383cb9d-sheet1.jpg?Expires=1689998324198&KeyName=labelbox-assets-key-3&Signature=HAtP_He0Xw_CARvC6cMmusNGHlk', 'details': {'dataset_id': 'clkc18z820elt07253e04gpkd', 'created_at': '2023-07-21T03:39:47.000+00:00', 'updated_at': '2023-07-21T03:39:47.000+00:00', 'last_activity_at': '2023-07-21T03:49:33.185+00:00', 'created_by': 'syukunt@gmail.com'}}, 'media_attributes': {'height': 1058, 'width': 748, 'mime_type': 'image/jpeg', 'exif_rotation': '1'}, 'attachments': [], 'metadata_fields': [], 'projects': {'clkc144x00eis07250u84cjom': {'name': 'CheckMarkDetection', 'labels': [{'label_kind': 'Default', 'version': '1.0.0', 'id': 'clkc1bszw0eob072502j80rn1', 'label_details': {'created_at': '2023-07-21T03:45:49.000+00:00', 'updated_at': '2023-07-21T03:45:49.000+00:00', 'created_by': 'syukunt@gmail.com', 'reviews': []}, 'p

In [69]:
# Define the transformations
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor()
])

# Create the dataset and data loader
dataset = CustomDataSet('../export-result.ndjson', transform=transform)
data_loader = DataLoader(dataset, batch_size=4, shuffle=True)

In [70]:
# ネットワークを訓練
# Initialize the network and optimizer
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Train the network
for epoch in range(2):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        print(data)
        # get the inputs; data is a list of [inputs, labels]
        inputs, _, labels = data  # We ignore the bounding boxes for now

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

TypeError: Compose.__call__() takes 2 positional arguments but 3 were given